<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/Kaggle/2023Q2/Un_primer_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import json,re
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from preparando_datos import sum_into_column,split_and_sum

import pandas as pd
import matplotlib.pyplot as plt





In [3]:
df_val=pd.read_csv('dataset/testear.csv')
df_train=pd.read_csv('dataset/origen.csv')


In [57]:
# Seasons worked
for df in [df_train,df_val]:
    writers = df.groupby("directors").count()["Unnamed: 0"].to_dict()
    writers = split_and_sum(writers)
    df["directors_exp"] = df["directors"].apply(sum_into_column,args=(writers,))
    
for df in [df_val,df_train]:
    seasons = df.groupby("directors").count()["seasonNumber"].to_dict()
    seasons = split_and_sum(seasons)
    df["dir_seasonsOfExperience"] = df["directors"].apply(sum_into_column,args=(seasons,))

# episodes worked
for df in [df_val,df_train]:
    episodes = df.groupby("directors").count()["episodeNumber"].to_dict()
    episodes = split_and_sum(episodes)
    df["dir_episodesOfExperience"] = df["directors"].apply(sum_into_column,args=(episodes,))
    
# movies worked
for df in [df_val,df_train]:
    movies = df[df["titleType"] == "movie"].groupby("directors").count()["titleType"].to_dict()
    movies = split_and_sum(movies)
    df["dir_moviesOfExperience"] = df["directors"].apply(sum_into_column,args=(movies,))

# other xp than movies
for df in [df_val,df_train]:
    others = df[df["titleType"] != "movie"].groupby("directors").count()["titleType"].to_dict()
    others = split_and_sum(others)
    df["dir_othersOfExperience"] = df["directors"].apply(sum_into_column,args=(others,))

del seasons,episodes,movies,others,df

In [58]:
for df in [df_train,df_val]:
    writers = df.groupby("writers").count()["Unnamed: 0"].to_dict()
    writers = split_and_sum(writers)
    df["writers_exp"] = df["writers"].apply(sum_into_column,args=(writers,))

for df in [df_val,df_train]:
    seasons = df.groupby("writers").count()["seasonNumber"].to_dict()
    seasons = split_and_sum(seasons)
    df["writer_seasonsOfExperience"] = df["writers"].apply(sum_into_column,args=(seasons,))

for df in [df_val,df_train]:
    episodes = df.groupby("writers").count()["episodeNumber"].to_dict()
    episodes = split_and_sum(episodes)
    df["writer_episodesOfExperience"] = df["writers"].apply(sum_into_column,args=(episodes,))

for df in [df_val,df_train]:
    movies = df[df["titleType"] == "movie"].groupby("writers").count()["titleType"].to_dict()
    movies = split_and_sum(movies)
    df["writer_moviesOfExperience"] = df["writers"].apply(sum_into_column,args=(movies,))

for df in [df_val,df_train]:
    others = df[df["titleType"] != "movie"].groupby("writers").count()["titleType"].to_dict()
    others = split_and_sum(others)
    df["writer_othersOfExperience"] = df["writers"].apply(sum_into_column,args=(others,))

del seasons,episodes,movies,others,df

In [20]:
def parse_json(x):
    try:
        return json.loads(x)
    except json.JSONDecodeError:
        print(x)
        pattern = r"^'|(\w)'(\w*[,:])"
        replacement = r'\1"\2'
        x = re.sub(pattern, replacement, x)
        print(x)
        
        
        print(x)
        return json.loads(x)
for df in [df_train]:
    #turn production company into a list
    #fill nans with empty list
    df["production_companies"] = df["production_companies"].fillna("[]")
    #turn str list into list
    df["production_companies"] = df["production_companies"].apply(parse_json)

[{'name': 'Minerva Film SpA', 'id': 153}, {'name': 'Rieti Film', 'id': 12136}]
[{'name": 'Minerva Film SpA", 'id": 153}, {'name": 'Rieti Film", 'id": 12136}]
[{'name": 'Minerva Film SpA", 'id": 153}, {'name": 'Rieti Film", 'id": 12136}]


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [60]:
# turn video colunm into 1 or 0
# video column might have False, True or NaN values
for df in [df_val,df_train]:
    df["video"] = df["video"].fillna(False)
    df["video"] = df["video"].astype(int)

# turn tagline into the lenth of the tagline
for df in [df_val,df_train]:
    df["tagline"] = df["tagline"].fillna("")
    df["tagline"] = df["tagline"].apply(len)

In [61]:
#set language to 0 if it is nan

df_train['language'] = df_train['language'].fillna(0)
df_val['language'] = df_val['language'].fillna(0)

In [62]:
y=df_train.averageRating
vars=['startYear', 'runtimeMinutes',"numVotes","writers_exp",
      "directors_exp","endYear","isAdult","seasonNumber",
      "episodeNumber","ordering","isOriginalTitle","runtime",
      "revenue","popularity","budget","dir_seasonsOfExperience",
      "dir_episodesOfExperience","dir_othersOfExperience","dir_moviesOfExperience",
      "writer_seasonsOfExperience","writer_episodesOfExperience","writer_othersOfExperience",
      "writer_moviesOfExperience","video","tagline"]
X=df_train[vars]
X = pd.concat([X, pd.get_dummies(df_train['language'], prefix='language')], axis=1)
X = pd.concat([X, pd.get_dummies(df_train['status'], prefix='status')], axis=1)
X_pred=df_val[vars]
X_pred = pd.concat([X_pred, pd.get_dummies(df_val['language'], prefix='language')], axis=1)
X_pred = pd.concat([X_pred, pd.get_dummies(df_val['status'], prefix='status')], axis=1)


In [63]:
for i,df in enumerate([df_train, df_val]):
    encoder = OneHotEncoder()
    title = encoder.fit_transform(df[['titleType']])
    title_df = pd.DataFrame(title.toarray(), columns=encoder.get_feature_names_out(['titleType']))
    #split by genres
    genres = df.genres_x.str.get_dummies(sep=',')
    genres = genres.reindex(sorted(genres.columns), axis=1)


    title_df = pd.concat([title_df, genres], axis=1)
    if not i:
        X = pd.concat([X, title_df], axis=1)
    else:
        X_pred = pd.concat([X_pred, title_df], axis=1)


for col in X.columns:
    if col not in X_pred.columns:
        X_pred[col] = 0


del title_df, genres, title, encoder, df





In [66]:
#drop duplicate columns
X = X.loc[:,~X.columns.duplicated()]
X_pred = X_pred.loc[:,~X_pred.columns.duplicated()]
print(X.columns)
#add missing columns
for col in X.columns:
    if col not in X_pred.columns:
        X_pred[col] = 0
for col in X_pred.columns:
    if col not in X.columns:
        X[col] = 0

Index(['0', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show',
       'Thriller', 'War', 'Western', 'budget', 'dir_episodesOfExperience',
       'dir_moviesOfExperience', 'dir_othersOfExperience',
       'dir_seasonsOfExperience', 'directors_exp', 'endYear', 'episodeNumber',
       'isAdult', 'isOriginalTitle', 'language_0', 'language_en',
       'language_es', 'language_fr', 'language_haw', 'language_hi',
       'language_myv', 'language_yi', 'numVotes', 'ordering', 'popularity',
       'revenue', 'runtime', 'runtimeMinutes', 'seasonNumber', 'startYear',
       'status_In Production', 'status_Planned', 'status_Post Production',
       'status_Released', 'status_Rumored', 'tagline', 'titleType_movie',
       'titleType_short', 'titleType_tvEpisod

In [65]:
X_pred = X_pred.sort_index(axis=1)
X = X.sort_index(axis=1)
X.to_csv('dataset/train.csv', index=False)
X_pred.to_csv('dataset/val.csv', index=False)